# 🌍 LangGraph Example: Travel Assistant with Weather & Historical Fact

This notebook demonstrates the use of external APIs:

- Accepts a user query (e.g., "I'm going to Paris")
- Extracts a city name
- Fetches temperature of the city from Open-Meteo API
- Retrieves a fact about the city from Wikipedia API
- Summarizes both using an LLM

In [27]:
# ✅ Install required packages
!pip install -q langchain openai langgraph requests

from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END

In [16]:
# ---------------------------------------------
# Step 1: Set up OpenAI.
# ----------------------------------------------

import os
from dotenv import load_dotenv
import json

# Load environment variables from .env file
load_dotenv()

# Check that the key is loaded
assert os.getenv("OPENAI_API_KEY") is not None, "OPENAI_API_KEY is not set"

# Initialize LLM using env key
llm = ChatOpenAI(temperature=0.3)

In [ ]:
# ---------------------------------------------
# Step 2: Define the state: a TypedDict.
# ----------------------------------------------
from typing import TypedDict

class State(TypedDict):
    user_query: str      # The prompt that invokes this graph
    city: str            # The prompt identifies a city of town
    weather: str         # The temperature in the city obtained from an API
    fact: str            # A fact about the city obtained from an API
    final_summary: str   # The output: a summary of the weather and fact

In [ ]:
# 🌐 External API tools
import requests
'''
Define two functions that interact with external APIs to 
fetch temperature and a fact about a city.

'''


def get_temperature(city: str) -> str:
    geo = requests.get(f"https://nominatim.openstreetmap.org/search?format=json&q={city}",
                       headers={"User-Agent": "langchain-agent"}).json()
    lat, lon = geo[0]["lat"], geo[0]["lon"]
    weather = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
    ).json()
    temp = weather["current_weather"]["temperature"]
    return f"The current temperature in {city} is {temp}°C."


def get_fact(city: str) -> str:
    city_url = city.replace(" ", "_")
    r = requests.get(
        f"https://en.wikipedia.org/api/rest_v1/page/summary/{city_url}"
    )
    if r.status_code != 200:
        return f"No Wikipedia summary found for {city}"
    return r.json().get("extract", "No summary available.")

In [24]:
# ---------------------------------------------
# Step 3: Specify the functions that are executed
# by nodes in the graph.
# The functions return a dict where the keys are
# also keys of State.
# ----------------------------------------------

def extract_city_function(state: State) -> dict:
    user_query = state["user_query"]
    if not user_query:
        raise ValueError("User query is required.")

    prompt = f"""
    You are a helpful assistant who can understand complex and 
    ill-formed queries. Determine the city or town mentioned in
    {user_query}
    

    Return a JSON object like this:
    {{
    "city": "<city name or Unknown>",
    "fact": "<fact about the city>"
    }}

    Rules:
    - If no city is found, use "Unknown"
    - If no fact is found, use "Unknown"
    
    """

    # Send prompt to LLM
    response = llm.invoke(prompt).content.strip()

    try:
        parsed = json.loads(response)
    except json.JSONDecodeError:
        raise ValueError(f"Could not parse response: {response}")

    city = parsed.get("city", "Unknown")
    fact = parsed.get("fact", "Unknown")
    print(f"Extracted city: {city}, fact: {fact}")
    return {"city": city, "fact": fact}


def call_weather_tool_function(state: State) -> dict:
    weather = get_temperature(state["city"])
    return {"weather": weather}


def call_fact_tool_function(state: State) -> dict:
    fact = get_fact(state["city"])
    return {"fact": fact}


def summarize_function(state: State) -> dict:
    prompt = f"""You are a helpful assistant.
You were asked: '{state['user_query']}'

Here is the weather report of {state['city']}: {state['weather']}
Here is a fact about {state['city']}:: {state['fact']}

Give a 2-line summary of the weather report and fact.
Only summarize the information provided, do not add any additional information.
"""
    response = llm.invoke(prompt)
    return {"final_summary": response.content}

In [25]:
# ---------------------------------------------
# Step 4: Build the graph
# The nodes of the graph are agents that execute
# functions that read and write the state.
# ----------------------------------------------

# 4.1 Create builder
builder = StateGraph(State)

# 4.2 Add nodes to the graph.
# Give a name to the node and specify the function
# that will be executed by the node.
builder.add_node("extract_city_node", RunnableLambda(extract_city_function))
builder.add_node("fetch_weather_node",
                 RunnableLambda(call_weather_tool_function))
builder.add_node("fetch_fact_node", RunnableLambda(call_fact_tool_function))
builder.add_node("summarize_node", RunnableLambda(summarize_function))

# 4.3 Define the edges between nodes of the graph.
builder.add_edge("extract_city_node", "fetch_weather_node")
builder.add_edge("fetch_weather_node", "fetch_fact_node")
builder.add_edge("fetch_fact_node", "summarize_node")

# 4.4 Specify the entry and finish points of the graph.
builder.set_entry_point("extract_city_node")
builder.set_finish_point("summarize_node")

# 4.5 Compile the graph
graph = builder.compile()

In [26]:
# 🚀 Run the assistant
user_input = "I am thinking of visiting museums in Pasadena"
initial_state = {"user_query": user_input}

result = graph.invoke(initial_state)
print(result["final_summary"])

Extracted city: Pasadena, fact: Pasadena is known for its cultural institutions, art galleries, and museums, including the Norton Simon Museum and the Pasadena Museum of California Art.
The current temperature in Pasadena is 23.4°C. Pasadena is a city in Los Angeles County, California, known for being the primary cultural center of the San Gabriel Valley.
